# Imports

In [1]:
import panel as pn
import sympy as sp
from sympy.parsing.latex import parse_latex        #Converting Latex input to Sympy
import re
pn.extension('katex')

# Widget Creation

In [2]:
#  Mode Selector (horizontal buttons)
radio_group = pn.widgets.RadioBoxGroup(name='Mode Selector', options=['None', 'Static', 'Monovariate', 'Bivariate'], inline=True, value='None')
mode = radio_group.value

#  Equation Entry
text_input = pn.widgets.TextInput(name='Equation Input', placeholder='Enter your LaTeX equation here...')

#  Latex Pane
latex = pn.pane.LaTeX('')

#  Variable Selection Dropdown
dep_drop = pn.widgets.Select(name='Dependent Variable', options=['No item selected'], visible=False)
indep_drop = pn.widgets.Select(name='Independent Variable', options=['No item selected'], visible=False)
indep_drop1 = pn.widgets.Select(name='Independent Variable 1', options=['No item selected'], visible=False)
indep_drop2 = pn.widgets.Select(name='Independent Variable 2', options=['No item selected'], visible=False)

## Update Latex Widget

In [3]:
def update_latex_view(event):
    equation = fr'${text_input.value}$'
    try:
        latex.object = equation
    except Exception as e:
        latex.object = "Invalid equation"
        print(f"Error parsing equation for LaTeX: {e}")

text_input.param.watch(update_latex_view, 'value')

Watcher(inst=TextInput(name='Equation Input', placeholder='Enter your LaTeX e...), cls=<class 'panel.widgets.input.TextInput'>, fn=<function update_latex_view at 0x0000027C4D91A3E0>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

## Simple Latex Updater

In [4]:
pn.Column(
    radio_group,
    text_input,
    latex
)

Column
    [0] RadioBoxGroup(inline=True, name='Mode Selector', options=['None', 'Static', ...], value='None')
    [1] TextInput(name='Equation Input', placeholder='Enter your LaTeX e...)
    [2] LaTeX(str)

## Dynamic Variable Appearance

In [5]:
variables_inputs = {}

# Define a function to update visibility based on the equation input
def update_vars(event):
    equation = event.new
    # Find all variable names in the equation (letters followed by optional digits or underscores)
    variables = re.findall(r'(?<!\\)\b[a-zA-Z_]\w*\b', equation)
    
    # Determine which variables are no longer needed and remove their widgets
    for var in list(variables_inputs):
        if var not in variables:
            del variables_inputs[var]
    
    # Create new widgets for new variables
    for var in variables:
        if var not in variables_inputs:
            variables_inputs[var] = pn.widgets.TextInput(name=var, visible=True, min_width=80, width_policy='max', width = 80, max_width = 80)
    
    # Update the layout with the new set of widgets
    layout[:] = [radio_group, text_input, latex] + list(variables_inputs.values())

# Attach the update function to the equation input widget
text_input.param.watch(update_vars, 'value')

# Layout the widgets in a column
layout = pn.Column(radio_group, text_input, latex)

layout

Column
    [0] RadioBoxGroup(inline=True, name='Mode Selector', options=['None', 'Static', ...], value='None')
    [1] TextInput(name='Equation Input', placeholder='Enter your LaTeX e...)
    [2] LaTeX(str)

## Variable Selection Dropdown

In [11]:
#This function changes the visibility of the dropdown widgets based on radiobox input
def drop_vis(event):
    if event.new == 'Static':
        dep_drop.visible = True
        indep_drop.visible = False
        indep_drop1.visible = False
        indep_drop2.visible = False
    elif event.new == 'Monovariate':
        dep_drop.visible = True
        indep_drop.visible = True
        indep_drop1.visible = False
        indep_drop2.visible = False
    elif event.new == 'Bivariate':
        dep_drop.visible = True
        indep_drop.visible = False
        indep_drop1.visible = True
        indep_drop2.visible = True

    # Clear previous selections
    dep_drop.value = 'No item selected'
    indep_drop.value = 'No item selected'
    indep_drop1.value = 'No item selected'
    indep_drop2.value = 'No item selected'

radio_group.param.watch(drop_vis, 'value')


def update_drop(event):
    equation = event.new
    # Find all variable names in the equation (letters followed by optional digits or underscores)
    variables = re.findall(r'(?<!\\)\b[a-zA-Z_]\w*\b', equation)
    # Setting new options
    dep_drop.options = variables
    indep_drop.options = variables
    indep_drop1.options = variables
    indep_drop2.options = variables

    # Clear previous selections
    dep_drop.value = 'No item selected'
    indep_drop.value = 'No item selected'
    indep_drop1.value = 'No item selected'
    indep_drop2.value = 'No item selected'

    update_vars(None)

text_input.param.watch(update_drop, 'value')


variables_inputs = {}
# Define a function to update visibility based on the equation input
def update_vars(event):
    equation = text_input.value
    mode = radio_group.value
    dep_var = dep_drop.value
    indep_var = indep_drop.value
    indep_var1 = indep_drop1.value
    indep_var2 = indep_drop2.value

    # Find all variable names in the equation (letters followed by optional digits or underscores)
    variables = re.findall(r'(?<!\\)\b[a-zA-Z_]\w*\b', equation)
    print(f"Found variables: {variables}")   #Debugging
    
    # Determine which variables are no longer needed and remove their widgets
    for var in list(variables_inputs):
        if var not in variables:
            del variables_inputs[var]
                
    # Remove values from those chosen as independent and dependent
    filtered_vars = [var for var in variables if var not in (dep_var, indep_var, indep_var1, indep_var2)]
    print(f"Filtered variables: {filtered_vars}")  # Debugging
    
    # Create new widgets for new variables
    for var in filtered_vars:
        if var not in variables_inputs:
            variables_inputs[var] = pn.widgets.TextInput(name=var, visible=True, min_width=80, width_policy='max', width = 80, max_width = 80)

    # Update the layout with the new set of widgets
    layout[:] = [radio_group, text_input, latex, dep_drop, indep_drop, indep_drop1, indep_drop2] + list(variables_inputs.values())

#Binding variable lists to other widgets
text_input.param.watch(update_vars, 'value')
dep_drop.param.watch(update_vars, 'value')
indep_drop.param.watch(update_vars, 'value')
indep_drop1.param.watch(update_vars, 'value')
indep_drop2.param.watch(update_vars, 'value')


# Layout the widgets in a column
layout = pn.Column(radio_group, text_input, latex, dep_drop, indep_drop, indep_drop1, indep_drop2)

layout

Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered variables: ['V', 'n', 'R', 'T'] 
Found variables: ['P', 'V', 'n', 'R', 'T'] 
Filtered v

Column
    [0] RadioBoxGroup(inline=True, name='Mode Selector', options=['None', 'Static', ...], value='Monovariate')
    [1] TextInput(name='Equation Input', placeholder='Enter your LaTeX e..., value='P*V = n*R*T', value_input='P*V = n*R*T')
    [2] LaTeX(str)
    [3] Select(name='Dependent Variable', options=['P', 'V', 'n', 'R', 'T'], value='P')
    [4] Select(name='Independent Variable', options=['P', 'V', 'n', 'R', 'T'], value='P')
    [5] Select(name='Independent Variable 1', options=['P', 'V', 'n', 'R', 'T'], value='P', visible=False)
    [6] Select(name='Independent Variable 2', options=['P', 'V', 'n', 'R', 'T'], value='P', visible=False)

In [12]:
equation = text_input.value
mode = radio_group.value
dep_var = dep_drop.value
indep_var = indep_drop.value
indep_var1 = indep_drop1.value
indep_var2 = indep_drop2.value

variables = re.findall(r'(?<!\\)\b[a-zA-Z_]\w*\b', equation)
print(f"Found variables: {variables}")   #Debugging

filtered_vars = [var for var in variables if var not in (dep_var, indep_var, indep_var1, indep_var2)]
print(f"Filtered variables: {filtered_vars}")  # Debugging
print(dep_var, indep_var, indep_var1, indep_var2) # Debugging

Found variables: ['P', 'V', 'n', 'R', 'T']
Filtered variables: ['V', 'n', 'R', 'T']
P P P P


In [8]:
string = r'$3\frac{\pi^2}{6} + a_n = 4$'
string = string[1:-1]
spstring = parse_latex(string)
spstring

Eq(a_{n} + 3*(pi**2/6), 4)

In [9]:
print(indep_drop.value)

No item selected
